<a href="https://colab.research.google.com/github/amritavarshini04/Genai_projects/blob/main/Customer_Feedback_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data={
    "review_id" : [1,2,3],
    "review_text" : [
        "Got it delivered quick and the product is amazing!",
        "The service was not good, and I got a wrong item.",
        "Okay product, but the packaging was not very good."
    ]
}
df=pd.DataFrame(data)
df.head()

,review_id,review_text
0,1,Got it delivered quick and the product is amaz...
1,2,"The service was not good, and I got a wrong item."
2,3,"Okay product, but the packaging was not very g..."


Run sentimental analysis, the hugging face

In [ ]:
from transformers import pipeline

sentiment_pipeline=pipeline("sentiment-analysis")
df["sentiment"]=df["review_text"].apply(lambda x: sentiment_pipeline(x)[0])
df.head()

Summarize long feedback

In [ ]:
summarizer = pipeline("summarization")
df['summary']=df['review_text'].apply(lambda x: summarizer(x, max_length=30, min_length=5, do_sample=False)[0]['summary_text'])
df.head()

Generating AI replies using Gemini API

In [ ]:
import google.generativeai as genai
from google.colab import userdata

try:
  API_KEY=userdata.get('GOOGLE_API_KEY')
  genai.configure(api_key=API_KEY)
  model=genai.GenerativeModel('gemini-pro')
except Exception as e:
  print(f"Error configuring Gemini API: {e}")
  print("Please make sure you have added your API key to Colab secrets with the name 'GOOGLE_API_KEY'.")

def generate_reply(review):
  if 'model' not in locals():
    return "Error: Gemini API not configured."
  prompt = f"You are a customer care agent. Respond politely to this customer feedback: {review}"
  try:
    response = model.generate_content(prompt)
    return response.text
  except Exception as e:
    return f"Error generating reply: {e}"

df['ai_reply'] = df['review_text'].apply(generate_reply)
df.head()

Visualize results, in streamlit dashboard

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt

st.title("Customer Feedback Analysis")

# Count sentiment labels
sentiment_counts = df['sentiment'].apply(lambda x: x['label']).value_counts()

# Display sentiment counts as a bar chart
st.bar_chart(sentiment_counts)

st.subheader("Individual Reviews and AI Replies")
for index, row in df.iterrows():
  st.write(f"**Review ID:** {row['review_id']}")
  st.write(f"**Review:** {row['review_text']}")
  st.write(f"**Sentiment:** {row['sentiment']['label']} (Score: {row['sentiment']['score']:.2f})")
  st.write(f"**Summary:** {row['summary']}")
  st.write(f"**AI Reply:** {row['ai_reply']}")
  st.write("---")